In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from lifelines import CoxPHFitter

In [2]:
data = pd.read_csv("data/merged_dataset_python.csv", index_col=0)

In [3]:
data["is_maried"] = data["is_maried"].astype(int)

In [4]:
train, test = train_test_split(data, random_state=100500, test_size=0.95)
#del(data)

In [5]:
# Exclude all fields except mentioned here
usefields = ["is_maried", "age", "work_experience", "children_number", "has_mortgage",
            "loan_products_nii", "checking_account_balance", "deposit_balance", "money_market_balance",
            "investable_asset_indicator", "Sentiment", "Sentiment_days_passed", 
            "Day_Observed", "Event"]
train_fewer = train[usefields]

In [6]:
cf = CoxPHFitter()
cf.fit(train_fewer, duration_col='Day_Observed', event_col='Event')
#cf.print_summary() 

D:\bin\Anaconda2\lib\site-packages\lifelines\fitters\coxph_fitter.py:285: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  df.sort(duration_col, inplace=True)


<lifelines.CoxPHFitter: fitted with 50000 observations, 39034 censored>

In [7]:
import pickle    
with open('data/learned_cox_model.pkl','wb') as f:
  pickle.dump(cf, f)


## Going to play with data ##

In [ ]:
expect = cf.predict_expectation(test[:8])
expect

In [ ]:
pd.concat([cf.predict_expectation(test[:80]), data.loc[test[:80].index, ["Event", "Day_Observed"]] ], axis=1)

In [ ]:
from lifelines.utils import k_fold_cross_validation
cf_validation = CoxPHFitter()
scores = k_fold_cross_validation(cf_validation, train_fewer, "Day_Observed", event_col="Event", k=3)
print scores
print np.mean(scores)
print np.std(scores)